In [82]:
class PatientPhenotype:
    
    def __init__(self, eid, case, sex, yearBirth):
        
        self.eid = eid
        self.case = case
        self.sex = sex
        self.yearBirth = yearBirth
        
    def disp(self):
        print("eid = ",self.eid)
        

In [83]:
patientsPhen = {}

f = open('C:\\Users\\Antonis\\Documents\\GitHub\\diplwmatikh\\data\\phenotype.txt','r')
f.readline()
for line in f:
    patientsPhen[line.split()[0]] = PatientPhenotype(line.split()[0],line.split()[1],line.split()[2],line.split()[3])
        

In [84]:
len(patientsPhen)

5150